# Tutorial 3: Confluence

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-normalization/blob/main/notebooks/03-confluence.ipynb)

> "Confluence means the path doesn't matter — only the destination." — Varen Tholl

## The Church-Rosser Property

In Year 932, Varen Tholl proved a fundamental property of the Calculus of Inductive Constructions (EV-932-003):

**Confluence (Church-Rosser)**: If M reduces to N₁ and M reduces to N₂, then there exists N₃ such that both N₁ and N₂ reduce to N₃.

```
        M
       / \
      ↓   ↓
     N₁   N₂
      \   /
       ↓ ↓
        N₃
```

This means: no matter which reduction path you take, you can always rejoin.

In [ ]:
import pandas as pd

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

# Load confluence proofs dataset
conf_df = pd.read_csv(BASE_URL + "confluence_proofs.csv")

conf_df[['system', 'property', 'statement', 'proof_technique']].head(5)

## Why Confluence Matters

Confluence has two crucial implications:

### 1. Unique Normal Forms

If a term has a normal form, that normal form is *unique* (up to alpha-equivalence).

Proof: Suppose M has two normal forms N₁ and N₂. By confluence, they have a common reduct N₃. But normal forms can't reduce, so N₁ = N₃ = N₂.

In [ ]:
# Unique normal forms
#
# (λx.x)((λy.y)z)  can be reduced two ways:
#
#   Path 1: (λx.x)((λy.y)z) → (λy.y)z → z
#   Path 2: (λx.x)((λy.y)z) → (λx.x)z → z
#
# Both reach z — the unique normal form

example = {
    'term': '(λx.x)((λy.y)z)',
    'path1': ['(λx.x)((λy.y)z)', '(λy.y)z', 'z'],
    'path2': ['(λx.x)((λy.y)z)', '(λx.x)z', 'z'],
    'normal_form': 'z'
}

print("Path 1 (outer first):", ' → '.join(example['path1']))
print("Path 2 (inner first):", ' → '.join(example['path2']))
print(f"Both reach the unique normal form: {example['normal_form']}")

### 2. Evaluation Strategy Doesn't Affect Results

With confluence (and normalization), we can choose any evaluation strategy and get the same answer. Strategy becomes a matter of *efficiency*, not *correctness*.

In [ ]:
# Load reduction strategies
strat_df = pd.read_csv(BASE_URL + "reduction_strategies.csv")

# Compare strategies
strat_df[['strategy_name', 'deterministic', 'used_for']]

## The Diamond Property

The simplest form of confluence is the **diamond property**:

If M → N₁ and M → N₂ (single steps), then there exists N₃ such that N₁ → N₃ and N₂ → N₃.

```
        M
       / \
      ↓   ↓  (single steps)
     N₁   N₂
      \   /
       ↓ ↓   (single steps)
        N₃
```

The diamond property for single steps implies confluence for multi-step reduction.

In [ ]:
# Diamond lemma: key to proving confluence
#
# If single-step has diamond property, multi-step has confluence
#
# Proof idea: tile the diagram with diamonds
#
#     M
#    /|\
#   / | \
#  N1 N2 N3
#   \ | /
#    \|/
#     N4

print("Diamond → Confluence")
print("")
print("Step 1: M → N1 and M → N2")
print("Step 2: By diamond, both reach N3")
print("Step 3: Repeat for each divergence")
print("Result: All paths eventually join")

## Parallel Reduction

Beta-reduction doesn't have the diamond property directly. Consider:

```
(λx.x x)((λy.y)z)
```

- Path 1 (outer): `((λy.y)z)((λy.y)z)` — the argument is duplicated!
- Path 2 (inner): `(λx.x x)z`

From `((λy.y)z)((λy.y)z)`, we need two more steps to reach the common form `z z`.

Tholl's solution: **parallel reduction**, which can reduce multiple redexes at once.

In [ ]:
# Parallel reduction: reduce multiple redexes simultaneously
#
# From ((λy.y)z)((λy.y)z):
#   Parallel reduce both (λy.y)z at once
#   → z z
#
# Parallel reduction DOES have the diamond property!

parallel_example = {
    'term': '((λy.y)z)((λy.y)z)',
    'redexes': ['(λy.y)z  (first)', '(λy.y)z  (second)'],
    'parallel_result': 'z z',
    'steps_saved': 1
}

print(f"Term: {parallel_example['term']}")
print(f"Redexes: {parallel_example['redexes']}")
print(f"Parallel reduction: {parallel_example['parallel_result']}")

## The Takahashi Method

Tholl used the Takahashi method to prove confluence (EV-932-003):

1. Define **parallel reduction** M ⇒ N that can reduce multiple redexes
2. Show parallel reduction has the diamond property
3. Show beta-reduction is contained in parallel reduction
4. Conclude beta-reduction is confluent

In [ ]:
# The Takahashi method
proof_structure = pd.DataFrame({
    'step': [1, 2, 3, 4],
    'goal': [
        'Define parallel reduction ⇒',
        'Prove ⇒ has diamond property',
        'Show → ⊆ ⇒ ⊆ →*',
        'Conclude → is confluent'
    ],
    'key_insight': [
        'Reduce all redexes at once',
        'No duplication of work',
        'Parallel is between single and multi',
        'Diamond for ⇒ implies confluence for →'
    ]
})

proof_structure

## Subject Reduction

A related property is **subject reduction** (type preservation):

If Γ ⊢ M : A and M → N, then Γ ⊢ N : A.

This ensures reduction doesn't change the type. Combined with confluence, this means normal forms have the same type as the original term.

In [ ]:
# Subject reduction examples
subject_red = pd.DataFrame({
    'term': [
        '(λx:Nat.x) zero',
        'succ ((λx:Nat.x) zero)',
        'head (cons a nil)'
    ],
    'type': ['Nat', 'Nat', 'A'],
    'reduces_to': ['zero', 'succ zero', 'a'],
    'type_preserved': [True, True, True]
})

subject_red

## Confluence in Practice

Let's trace how confluence works with the confluence proofs dataset:

In [ ]:
# What properties did Tholl prove?
tholl_proofs = conf_df[conf_df['discovered_by'] == 'varen_tholl']

tholl_proofs[['system', 'property', 'proof_technique', 'difficulty']]

## Summary

| Property | Statement | Implication |
|----------|-----------|-------------|
| Diamond | Single steps can rejoin in one step | Proof technique |
| Confluence | Multi-step reductions can rejoin | Normal forms are unique |
| Subject reduction | Reduction preserves types | Types are stable |

Confluence ensures that computation is *deterministic* in its result, even if the process of computing can vary.

---

**Next Tutorial:** Reducibility Candidates — The technique for proving normalization